# MLOps 

This notebook is intended to help you get started managing your machine learning models and deployments in the H2O AI Cloud using python. 

**_This notebook assumes you have run 1 End to End Demo and have a project with a deployed model that exists with a specific name_**

* **Product Documentation:** https://docs.h2o.ai/mlops/
* **Python Documentation:** https://docs.h2o.ai/mlops/py-client/install

In [ ]:
import os
import requests
import json

import h2o_authn
import h2o_discovery
import h2o_mlops

import pandas as pd

## Connect to MLOps

In [ ]:
discovery = h2o_discovery.discover()

token_provider = h2o_authn.TokenProvider(
    refresh_token=os.getenv("H2O_CLOUD_CLIENT_PLATFORM_TOKEN"),
    issuer_url=discovery.environment.issuer_url,
    client_id=discovery.clients["platform"].oauth2_client_id,
)

In [ ]:
mlops = h2o_mlops.Client(
    gateway_url=discovery.services['mlops-api'].uri,
    token_provider=token_provider
)

## Projects

### Create a project

In [ ]:
project = mlops.projects.create(
    name="my_test_project",
    description="Demo project for a tutorial"
)

### List all projects you have access to

In [ ]:
mlops.projects.list()

### Delete a project 

In [ ]:
project.delete()

In [ ]:
mlops.projects.list()

### Select a specific project to work with
We have previously created this project using Driverless AI and added models to it in our first tutorial.

In [ ]:
project = mlops.projects.list(name="Telco Churn Predictions")[0]

## Experiments

### List experiments
Get a list of all experiments that are in our project

In [ ]:
project.experiments.list()

### Select an experiment

In [ ]:
experiment = project.experiments.get(uid="ca85ba4a-a01a-11ee-95af-425f7a7f3fbf")

## Models

### List models

Get a list of all models that are in our project

In [ ]:
project.models.list()

### Select a model

In [ ]:
model = project.models.get(uid="7737d493-83cd-46b6-a7e9-3f47008d359e")

## Deployments

### Deployment environment 
`DEV` and `PROD` are deployment environment tags that you can use for your model deployements. 

In [ ]:
environment = project.environments.list(name="DEV")[0]

### Deployment runtime

The scoring runtime is used to load and score a scoring artifact from a model.

In [ ]:
runtime = mlops.runtimes.scoring.list(
    artifact_type=model.get_experiment().artifact_types[0],
    uid="dai_mojo_runtime"
)[0]

## Deployment creation

In [ ]:
deployment = environment.deployments.create_single(
    name=model.name,
    model=model,
    scoring_runtime=runtime
)

### Deployment health
Wait until our deployment has gone from launching to healthy

In [ ]:
import time


while not deployment.is_healthy():
    deployment.raise_for_failure()
    time.sleep(5)

In [ ]:
print(deployment.status())
print(deployment.url_for_scoring)
print(deployment.get_sample_request())

## Make predictions
Using the `requests` library, we make predictions on new data

In [ ]:
predictions = requests.post(
    url=deployment.url_for_scoring,
    json=deployment.get_sample_request()
)

In [ ]:
predictions.json()

In [ ]:
pd.DataFrame(predictions.json()["score"], columns=predictions.json()["fields"])

## Delete a deployment

In [ ]:
deployment.delete()